In [6]:
import pandas as pd
import glob
import os
import keras

In [2]:
#save the models to model_folder
model_folder='F:/Research/EnergyEstimation/models/'
#output folder of trajactory got from the simulation
output_folder='E:/SUMO/RUIXIAO/newChattanooganet/Data/output/outtest/'
#create energy_folder to save the prediction results
energy_folder='F:/Research/EnergyEstimation/test/'

# Select scenario for weather condition

In [3]:
#weather option: rainy, sunny, windy, and snowy
weather = 'rainy'

# Load microscopic energy eatimation model for different vehicle classes


In [4]:
model_diesel =  keras.models.load_model(f'{model_folder}diesel.model')
model_hybrid =  keras.models.load_model(f'{model_folder}hybrid.model')
model_electric =  keras.models.load_model(f'{model_folder}electric.model')

# Predict energy consumption of each trips 

In [5]:
path = f'{output_folder}'                    
all_files = glob.glob(os.path.join(path, "Trajectory*.csv")) 
colnames=['time_ms','speed','acceleration','vehicle_ref','actorConfig_id','actorConfig_emissionClass','actorConfig_fuel','actorConfig_ref','actorConfig_vehicleClass']
for f in all_files:
    trip=pd.read_csv(f,names=colnames, header=None)
    trip = trip[trip.speed != 'speed']
    trip = trip[trip.acceleration != 'acceleration']
    trip['speed']=trip['speed'].astype(float)*0.036 #km/h
    #find the vehicle class of this trip based on the column "actorConfig_fuel"
    if trip['actorConfig_fuel'].iloc[2]=='Gasoline':
        Vclass = 'diesel'
    elif trip['actorConfig_fuel'].iloc[2]=='HybridGasoline':
        Vclass = 'hybrid'
    elif trip['actorConfig_fuel'].iloc[2]=='Electric':
        Vclass = 'electric'
    if Vclass == 'electric':
        trip['acceleration']=trip['acceleration'].astype(float)*0.001*3.28084
        #ft/s2
    else:
        trip['acceleration']=trip['acceleration'].astype(float)*0.001 
        #m/s2
    if weather == 'rainy':
        trip['temperature']= 60 
        trip['humidity']= 0.9 #range[0,1]
    if weather == 'sunny':
        trip['temperature']= 70
        trip['humidity']= 0.5
    if weather == 'windy':
        trip['temperature']= 60
        trip['humidity']= 0.6
    if weather == 'snowy':
        trip['temperature']= 30
        trip['humidity']= 0.4
    trip['roadGrade']= 0
    input2esti=trip[['speed','acceleration','temperature','humidity','roadGrade']]
    #prdiction 
    if Vclass == 'diesel':
        pre = model_diesel.predict(input2esti)
        tripf=pd.concat([trip,pd.DataFrame(pre,columns=['energy_'+Vclass])], axis=1) 
        #write the predicted results 
        with open(f'{energy_folder}' + Vclass +'_'+ f[-44:-4] + '.csv', 'w', newline='') as oFile:
            tripf.to_csv(oFile, index = False)
    elif Vclass == 'hybrid':
        pre = model_hybrid.predict(input2esti)
        tripf=pd.concat([trip,pd.DataFrame(pre,columns=['energy_'+Vclass])], axis=1) 
        #write the predicted results 
        with open(f'{energy_folder}' + Vclass +'_'+ f[-44:-4] + '.csv', 'w', newline='') as oFile:
            tripf.to_csv(oFile, index = False)
    elif Vclass == 'electric':
        pre = model_electric.predict(input2esti) 
        tripf=pd.concat([trip,pd.DataFrame(pre,columns=['energy_'+Vclass])], axis=1)
        #write the predicted results 
        with open(f'{energy_folder}' + Vclass +'_'+ f[-44:-4] + '.csv', 'w', newline='') as oFile:
            tripf.to_csv(oFile, index = False)
    